In [6]:
import csv
import pandas as pd

name = []
with open('name.csv', 'r') as csvfile:
    csvreader = csv.reader(csvfile)
    for row in csvreader:
        name.append(row)

res = []

with open('jeju_food.csv', 'r') as csvfile:
    csvreader = csv.reader(csvfile)

    for row in csvreader:
        if row[5].find('폐업') == -1:
            res.append([row[0], row[13]])
print(name)
result = []

# threshold = 0.6
for x in name:
    cnt = 0
    for item in res:
        if cnt >= 2:
            break
        # if item[0].find(x[0]) != -1:
        if item[0] == x[0]:
            cnt += 1
            new_res = [item[0], x[1], item[1]]
            result.append(new_res)
    if cnt == 0:
        result.append([x[0], x[1], ''])

df = pd.DataFrame(result, columns=['식당 이름', '횟수', '도로명 주소'])
df


[['한라산', '4'], ['더비치크랩', '10'], ['맘앤타르트', '3'], ['이츠버거', '7']]


,식당 이름,횟수,도로명 주소
0,한라산,4,"제주특별자치도 제주시 구좌읍 면수1길 48, 2동 1층"
1,더비치크랩,10,"제주특별자치도 서귀포시 안덕면 사계남로 168, 2층"
2,맘앤타르트,3,
3,이츠버거,7,


In [7]:
from geopy.geocoders import Nominatim

def geocoding(address):
    geolocoder = Nominatim(user_agent = 'South Korea', timeout=None)
    geo = geolocoder.geocode(address)
    if geo is not None:
        crd = [geo.latitude, geo.longitude]
        return crd
    else:
        return None

str_del1 = ','
str_del2 = '('

for i, (name, num, addr) in enumerate(result):
    if str_del1 in addr:
        addr = addr[:addr.index(str_del1)]
    if str_del2 in addr:
        addr = addr[:addr.index(str_del2)]
    else:
        pass
    result[i] = [name, num, addr]

for item in result:
    if geocoding(item[2]) is not None:
        lat = geocoding(item[2])[0]
        log = geocoding(item[2])[1]
        item.append(lat)
        item.append(log)

df1 = pd.DataFrame(result, columns=['식당 이름', '횟수', '도로명 주소', '위도', '경도'])
df1

,식당 이름,횟수,도로명 주소,위도,경도
0,한라산,4,제주특별자치도 제주시 구좌읍 면수1길 48,NaN,NaN
1,더비치크랩,10,제주특별자치도 서귀포시 안덕면 사계남로 168,33.22821,126.307761
2,맘앤타르트,3,,NaN,NaN
3,이츠버거,7,,NaN,NaN


In [8]:
rest_name = []
new_result = [] #NaN 값 없는 result
for e in result:
    if len(e) < 3:
        rest_name.append([e[0], e[1]])
    elif len(e) == 3:
        rest_name.append([e[0], e[1], e[2]])
    else:
        new_result.append(e)
print(rest_name)
rest_res = []

with open('jeju_market.csv', 'r') as csvfile:
    csvreader = csv.reader(csvfile)
    for row in csvreader:
        rest_res.append([row[0], row[12], row[14], row[15]])

rest_result =[]
for x in rest_name:
    cnt = 0
    for item in rest_res:
        # if item[0].find(x[0]) != -1:
        if len(x) < 3:
            if item[0] == x[0]:
                cnt += 1
                rest_result.append([item[0], x[1], item[1], item[2], item[3]])
        elif len(x) == 3:
            if item[0] == x[0] and x[2].find(item[1]) != -1:
                cnt += 1
                rest_result.append([item[0], x[1], x[2], item[2], item[3]])
    if cnt == 0:
        rest_result.append([x[0], x[1]])

all_result = new_result + rest_result
df2 = pd.DataFrame(all_result, columns=['식당 이름', '횟수', '도로명 주소', '위도', '경도'])
df2


[['한라산', '4', '제주특별자치도 제주시 구좌읍 면수1길 48'], ['맘앤타르트', '3', ''], ['이츠버거', '7', '']]


,식당 이름,횟수,도로명 주소,위도,경도
0,더비치크랩,10,제주특별자치도 서귀포시 안덕면 사계남로 168,33.22821,126.307761
1,한라산,4,제주특별자치도 제주시 구좌읍 면수1길 48,126.8629599,33.52471461
2,맘앤타르트,3,None,None,None
3,이츠버거,7,None,None,None


In [9]:
from selenium import webdriver
from selenium.webdriver import Keys
from selenium.webdriver.common.by import By
import time

import re

driver = webdriver.Chrome()
driver.implicitly_wait(3)

def switch_frame(frame):
    driver.switch_to.default_content()
    driver.switch_to.frame(frame)

def remove_after_number(str):
    result = re.sub(r'(\d)\s.*', r'\1', str)
    return result

crawl = []
for e in all_result:
    if len(e) < 3:
        crawl.append([all_result.index(e),e[0]])
print(crawl)
crawl_data = []

for r in range(len(crawl)):
    driver.get('https://map.naver.com/')
    time.sleep(3)
    search = driver.find_element(By.CSS_SELECTOR, 'div.input_box > input.input_search')
    search.send_keys(crawl[r][1])
    search.send_keys(Keys.ENTER)

    time.sleep(1)
    switch_frame('searchIframe')
    time.sleep(3)

    res_list = driver.find_elements(By.CSS_SELECTOR, 'li.VLTHu')
    print(f'{len(res_list)}개의 검색 결과가 나왔습니다.')
    for data in range(len(res_list)):
        res_addr = ''
        addr_button = driver.find_elements(By.CSS_SELECTOR, 'span.lWwyx > a')
        addr_button.__getitem__(data).click()
        time.sleep(2)
        addr = driver.find_elements(By.CSS_SELECTOR, 'div > div.AbTyi > div.zZfO1')
        road = addr.__getitem__(0).text
        road_addr = road[3:(len(road) - 2)]
        if road_addr.startswith('제주'):
            real_addr = remove_after_number(road_addr)
            lat = geocoding(real_addr)[0]
            log = geocoding(real_addr)[1]
            all_result[crawl[r][0]].append(real_addr)
            all_result[crawl[r][0]].append(lat)
            all_result[crawl[r][0]].append(log)
            break

    time.sleep(5)

df3 = pd.DataFrame(all_result, columns=['식당 이름', '횟수', '도로명 주소', '위도', '경도'])
df3

[[2, '맘앤타르트'], [3, '이츠버거']]
2개의 검색 결과가 나왔습니다.
6개의 검색 결과가 나왔습니다.


,식당 이름,횟수,도로명 주소,위도,경도
0,더비치크랩,10,제주특별자치도 서귀포시 안덕면 사계남로 168,33.22821,126.307761
1,한라산,4,제주특별자치도 제주시 구좌읍 면수1길 48,126.8629599,33.52471461
2,맘앤타르트,3,제주 제주시 애월읍 중산간서로 6008,33.454389,126.396585
3,이츠버거,7,제주 제주시 서광로29길 8,33.503408,126.52681


In [10]:
print(all_result)

[['더비치크랩', '10', '제주특별자치도 서귀포시 안덕면 사계남로 168', 33.2282098, 126.3077609], ['한라산', '4', '제주특별자치도 제주시 구좌읍 면수1길 48', '126.8629599', '33.52471461'], ['맘앤타르트', '3', '제주 제주시 애월읍 중산간서로 6008', 33.4543891, 126.3965845], ['이츠버거', '7', '제주 제주시 서광로29길 8', 33.5034082, 126.5268105]]
